In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

%matplotlib inline
from datetime import datetime
from neo4j.exceptions import ServiceUnavailable
import humanize
import os
import re
import sys
import time
import warnings
import winsound

warnings.filterwarnings('ignore')
duration = 1000  # milliseconds
freq = 880  # Hz

# Insert at 1, 0 is the script path (or '' in REPL)
if (osp.join('..', 'py') not in sys.path): sys.path.insert(1, osp.join('..', 'py'))

In [3]:

# Get the Storage object
from storage import Storage
s = Storage(
    data_folder_path=os.path.abspath('../data'),
    saves_folder_path=os.path.abspath('../saves')
)

# Get the WebScrapingUtilities object
from scrape_utils import WebScrapingUtilities
wsu = WebScrapingUtilities(
    s=s,
    secrets_json_path=os.path.abspath('../data/secrets/jh_secrets.json')
)
uri = wsu.secrets_json['neo4j']['connect_url']
user =  wsu.secrets_json['neo4j']['username']
password = wsu.secrets_json['neo4j']['password']

# Get the HeaderAnalysis object
from ha_utils import HeaderAnalysis
ha = HeaderAnalysis(s=s, verbose=False)

# Get the CypherUtilities object and Neo4j driver
from cypher_utils import CypherUtilities
cu = CypherUtilities(
    uri=uri, user=user, password=password, driver=None, s=s, ha=ha
)

In [4]:

def do_cypher_tx(tx, file_name):
    cypher_str = """
        MATCH (fn:FileNames)
        WHERE fn.file_name = $file_name
        RETURN fn.percent_fit AS percent_fit;"""
    
    return [dict(record.items()) for record in tx.run(query=cypher_str, parameters={'file_name': file_name})]

In [5]:

# Get the scraped job posting file names
files_list = os.listdir(cu.SAVES_HTML_FOLDER)

# Filter in just the Accenture files
name_regex = re.compile(r'[3-4][0-9]{6}_[a-zA-Z0-9_]+\.html')
for file_name in files_list:
    if name_regex.fullmatch(file_name):
        
        # Get the % fit from the node
        percent_fit = 1.0
        with cu.driver.session() as session:
            percent_fit = session.write_transaction(do_cypher_tx, file_name=file_name)[0]['percent_fit']
        
        # If the percent fit is less than two thirds
        if percent_fit < 2/3:
            
            # Remove the node features and folder storage
            cu.delete_filename_node(file_name, remove_node=True, remove_file=True, verbose=True)


                    MATCH (fn:FileNames {file_name: "4738495_MARRIOTT_INTERNATIONAL_Program_and_Project_Management_Manager_Payments_Copy_01.html"})
                    DETACH DELETE fn;
C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\html\4738495_MARRIOTT_INTERNATIONAL_Program_and_Project_Management_Manager_Payments_Copy_01.html removed
